## For this notebook, we assume that the spectra extracted with "optimal" mode and also by "uniform" mode.
## It is also required that "combined_image" is generated by running the pipeline in "debug" mode

> python igr_pipe.py a0v-ab 20141023 -b K -s 80 -d --lacosmics-thresh=2 --weighting-mode="uniform" --basename-postfix="_uniform" -d

> python igr_pipe.py a0v-ab 20141023 -b K -s 80 -d --lacosmics-thresh=2 --basename-postfix="_optimal" -d

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("..")

import igrins
from igrins.qa.plots import imshow

In [ ]:
band = "K"
utdate = 20141023

In [ ]:
caldb = igrins.get_caldb("../recipe.config", utdate)
recipe_log = igrins.get_recipe_log(caldb)

In [ ]:
log_a0v = recipe_log.subset(recipe="A0V_AB")
log_a0v

In [ ]:
mylog = log_a0v.loc[8] # select by index number

obsset = igrins.get_obsset_from_log(caldb, band, mylog)

In [ ]:
hdu = obsset.load_item("combined_image", basename_postfix="_optimal")
hdu.info()

In [ ]:
bias_mask = obsset.load_resource_for("bias_mask")[0].data


In [ ]:
d = np.ma.array(hdu[0].data, mask=~bias_mask).filled(np.nan)

In [ ]:
fig = plt.figure()
im = imshow(fig, d, zscale=True, cmap="bwr")
fig.colorbar(im)

In [ ]:
omap = obsset.load_resource_for("ordermap")[0].data
slitpos = obsset.load_resource_for("slitposmap")[0].data
orders = obsset.get("orders")

In [ ]:
# invert the negative part

cor = np.ones(omap.shape, dtype="i")
cor[slitpos > 0.5] = -1
d1 = d*cor

fig = plt.figure()
im = imshow(fig, d1, zscale=True)
fig.colorbar(im)

## extract the spectra by naively summing the values along the y-direction

In [ ]:
s_list = []

for o in orders:
    m = omap == o
    _dm = np.ma.array(d1, mask=~m).filled(np.nan) # fill NaN for pixels outside of the given order
    ss = np.nansum(_dm, axis=0)
    s_list.append(ss)



In [ ]:
s_opt = obsset.load_item("spec_fits", basename_postfix="_optimal")[0].data
s_uni = obsset.load_item("spec_fits", basename_postfix="_uniform")[0].data

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

for ax, i in zip([ax1, ax2], [8, 14]):
    ax.plot(s_list[i], label="naive sum")
    ax.plot(s_opt[i], label="optimal")
    ax.plot(s_uni[i], label="uniform")
    smax = np.nanpercentile(s_opt[i], 95)*1.1
    ax.set_ylim(0, smax)

ax1.legend()

In [ ]:
from igrins.qa.plotly import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from igrins.qa.plotly.spec import figure_frame_data


In [ ]:
figure = figure_frame_data(orders,
                           [s_list, s_opt, s_uni],
                           name_list=["naive", "optimal", "uniform"])
iplot(figure)

## Check if a flux is conserved in the rectified spec2d images.

In [ ]:
spec2d_hdu = obsset.load_item("spec2d_fits")
spec2d_hdu.info()

In [ ]:
fig = plt.figure()
vv = np.nanpercentile(np.abs(spec2d_hdu[0].data[10]), 99.5)
imshow(fig, spec2d_hdu[0].data[10], aspect="auto", cmap="bwr", vmin=-vv, vmax=vv)

In [ ]:
s_rect = []
cor = np.ones(spec2d_hdu[0].data[0].shape, dtype="i")
cor[31:, :] = -1


In [ ]:
for rect in spec2d_hdu[0].data:
    ss = np.nansum(rect*cor, axis=0)
    s_rect.append(ss)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

for ax, i in zip([ax1, ax2], [8, 14]):
    ax.plot(s_uni[i], label="uniform")
    ax.plot(s_rect[i], label="rect")
    smax = np.nanpercentile(s_opt[i], 95)*1.1
    ax.set_ylim(0, smax)

ax1.legend()

In [ ]:
figure = figure_frame_data(orders,
                           [s_uni, s_rect],
                           name_list=["uniform", "rect"])
iplot(figure)